In [2]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

In [37]:
H, W = 512,512
cc = 384

In [3]:
#read the data
def load_data(path):
    images = sorted(glob(os.path.join(path, "image_1/*")))
    masks = sorted(glob(os.path.join(path, "mask_1/*")))
    return images, masks

In [4]:
#create folder for save augmented images
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [41]:
#augmentation
def augment_data(images, masks, save_path, augment=True):

    for x, y in tqdm(zip(images, masks), total=(len(images))):
        name = x.split("\\")[-1].split(".")
        image_name = name[0]
        image_ext = name[1]
        
        name = y.split("\\")[-1].split(".")
        mask_name = name[0]
        mask_ext = name[1]
        
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        
        if augment == True:
            aug = CenterCrop(cc, cc, p=1.0)
            augemented = aug(image=x, mask=y)
            x1 = augemented["image"]
            y1 = augemented["mask"]
            
            aug = RandomRotate90(p=1.0)
            augemented = aug(image=x, mask=y)
            x2 = augemented["image"]
            y2 = augemented["mask"]
            
            save_images = [x, x1, x2]
            save_masks = [y, y1, y2]
        else:
            save_images = [x]
            save_masks = [y]
        
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            
            temp_img_name = f"{image_name}_{idx}.{image_ext}"
            temp_mask_name = f"{mask_name}_{idx}.{mask_ext}"
            
            image_path = os.path.join(save_path, "images", temp_img_name)
            mask_path = os.path.join(save_path, "masks", temp_mask_name)
            
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)
            idx += 1

In [6]:
path = "C:\\Users\Dark-Devil\\Desktop\\skull_strpping - Copy\\train"

In [7]:
images, masks = load_data(path)

In [8]:
print(f'Original Images : {len(images)} - Original masks: {len(masks)}')

Original Images : 722 - Original masks: 722


In [9]:
create_dir("train/images")
create_dir("train/masks")

In [12]:
save_path = "train"

In [42]:
augment_data(images, masks, save_path, augment=True)

100%|████████████████████████████████████████████████████████████████████████████████| 722/722 [01:09<00:00, 10.39it/s]
